In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### **A. MAB agent formulation :**

##### The problem agent formulation involves determining the most optimal ad to display to a user at a given time instant to maximize the number of clicks on the webpage. 

#### The problem can be defined as :

 1. There are 10 different ads to choose from, and at each time step, the MAB agent must decide which ad to display to the user. 

 2.  Goal is to maximize the total number of clicks obtained from the users over a specified time horizon. 

 3.  Each ad has an unknown click-through rate (CTR) that represents the probability of a user clicking on that ad. 
 
 4.  The MAB agent's objective is to learn the true CTR of each ad while minimizing the regret, which is the difference between the expected number of clicks obtained by displaying the best ad and the expected number of clicks obtained by displaying the chosen ad at each time step. 
 
 5.  The MAB agent must balance the exploration of less-known ads to learn their CTRs with the exploitation of the ads that are known to have higher CTRs to maximize the total number of clicks.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random

In [ ]:
ads_clicks = pd.read_csv('/content/drive/MyDrive/Artificial Intelligence Lab Sem - VI/Lab 6/Ads_Optimisation.csv')

In [ ]:
num_ads = len(ads_clicks.columns)
print(num_ads)

10


### **B. Total rewards after 2000-time steps using the ε-greedy algorithm for**

 1.  ε=0.01

2.  ε= 0.3

In [ ]:
# ε-greedy algorithm

def epsilon_greedy(epsilon, rewards):
    if random.uniform(0, 1) < epsilon:
        # Explore: Choose a random ad
        ad = random.randint(0, num_ads - 1)
    else:
        # Exploit: Choose the ad with the highest reward
        ad = np.argmax(rewards)
    return ad

In [ ]:
# Initialize the rewards for each ad to 0 and create an empty list to store the rewards for each time step:

rewards = [0] * num_ads
total_rewards_01 = []
total_rewards_03 = []

In [ ]:
# Iterating the ε-greedy algorithm for 2000 time steps using ε=0.01 and ε=0.3

for t in range(2000):

    # Choosing ad using the epsilon-greedy algorithm with epsilon=0.01
    ad_01 = epsilon_greedy(0.01, rewards)

    # Choose the ad using the epsilon-greedy algorithm with epsilon=0.3
    ad_03 = epsilon_greedy(0.3, rewards)
    
    # for epsilon = 0.01
    # reward for the chosen ad
    reward = ads_clicks.iloc[t][ad_01]
    # Updating rewards for the chosen ad
    rewards[ad_01] = rewards[ad_01] + reward
    # Add the reward to the total rewards list for epsilon=0.01
    total_rewards_01.append(sum(rewards))
    
    # for epsilon = 0.3
    # reward for the chosen ad
    reward = ads_clicks.iloc[t][ad_03]
    # Updating rewards for the chosen ad
    rewards[ad_03] = rewards[ad_03] + reward
    # Add the reward to the total rewards list for epsilon=0.3
    total_rewards_03.append(sum(rewards))

In [ ]:
print("Total rewards for ε=0.01: ", total_rewards_01[-1])
print("Total rewards for ε=0.3: ", total_rewards_03[-1])

Total rewards for ε=0.01:  642
Total rewards for ε=0.3:  642


### **C. Compute the total rewards after 2000-time steps using the Upper-Confidence-Bound action method for c = 1.5**

In [ ]:
# Upper-Confidence-Bound algorithm

def ucb(rewards, n, t, c=1.5):
    # Calculate the average reward for each ad
    average_rewards = rewards / n
    # Calculate the upper confidence bound for each ad
    ucb_values = average_rewards + c * np.sqrt(np.log(t + 1) / n)
    # Choose the ad with the highest UCB value
    ad = np.argmax(ucb_values)
    return ad

In [ ]:
# Initialize the rewards for each ad to 0 and create an empty list to store the rewards for each time step:

rewards = np.zeros(num_ads)
n = np.zeros(num_ads)
total_rewards = []

In [ ]:
# Iterating over Upper-Confidence-Bound algorithm for 2000 time steps using c=1.5:

for t in range(2000):
    # Choose the ad using the UCB algorithm
    ad = ucb(rewards, n, t, c=1.5)
    
    # Get the reward for the chosen ad
    reward = ads_clicks.iloc[t][ad]
    # Update the rewards for the chosen ad
    rewards[ad] = rewards[ad] + reward
    # Update the number of times the ad has been selected
    n[ad] = n[ad] + 1
    # Add the reward to the total rewards list
    total_rewards.append(sum(rewards))

# Print the total rewards for c=1.5
print("\nTotal rewards for c=1.5: ", total_rewards[-1])


Total rewards for c=1.5:  763.0


### **D. For all approaches, explain how the action value estimated compares to the optimal action.**

> ### How the action value estimated compares to the optimal action in the ε-greedy approach :

In the ε-greedy approach, action value is estimated using the **sample average method**, where the "**estimated value of each action = average of the rewards received for that action**". 

However, the sample average method can give large variance in the estimate if the number of samples is small. Thereby, the estimates may not converge to the true action values quickly. Hence, the ε-greedy approach tends to explore more at the beginning of the experiment to reduce uncertainty, and then exploit the best action based on the estimated action values later on.

> ### How the action value estimated compares to the optimal action in the UCB approach :

The Upper-Confidence-Bound approach, on the other hand, estimates the action values using the **Upper Confidence Bound (UCB)**, which is a **combination of the average reward and an uncertainty term**. 

The uncertainty term ensures that actions that have not been selected many times are given a higher priority to be selected, while actions that have been selected many times are given a lower priority. This approach results in a more stable estimate of the action values, and the algorithm converges more quickly to the optimal action.

In UCB, the algorithm tends to explore less often as it becomes more confident about the rewards of each arm. If the UCB algorithm becomes too confident too quickly, it may miss out on better rewards that could be obtained by exploring more.

On the other hand, the epsilon-greedy algorithm always explores with a fixed probability, regardless of how confident it is about the rewards of each arm. This can lead to more exploration and potentially better rewards.